To do:
- merge valid and train
- generate new valid: https://www.kaggle.com/anqitu/data-distribution-eda-and-leaderboard-exploit/code
- train linear classifier and compare to MLP performance (drop these features if there is no difference in performance)
- try getting activations from earlier in the network
- transfer learning: adjust later last layers of ResNet50 to our task
- train CNN from scratch: http://cs231n.github.io/transfer-learning/
- check scaling

In [1]:
import numpy as np
# import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns
import json
import csv
import time
from copy import deepcopy

%matplotlib inline 

import os
print(os.listdir("."))
import glob
import pickle

FEATURES_LOCATION = '/home/1nnuser/furniture-data/features_densenet/'
F_CORE = 'cnn_features_'

def get_label_from_path(file):
    return file.split('\\')[1].split('.')[0]

def load_data(mode):
    if(mode == 'test'):
        pickle_path = F_CORE + mode
        data = pickle.load(open(FEATURES_LOCATION + pickle_path + '.pkl', 'rb'))
        to_return = {}
        for key, value in list(data.items()):
            to_return[get_label_from_path(key)] = value.reshape(1,-1)
        return to_return, None
    
    pickle_path = F_CORE + mode + '_'
    
    data = {}
    for i in range(1,129):
        print("Pickle:", i, end='\r')
        data[i] = pickle.load(open(FEATURES_LOCATION + pickle_path + str(i) + '.pkl', 'rb'))
        
    X = []
    y = []
    for key, value in list(data.items()):
        the_class = key
        features = np.array(list(value.values()))
        for feature in features:
            y.append(the_class)
            X.append(feature)

    return np.array(X), np.array(y)

def get_id_from_path(path):
    return int(path.split('/')[-1].split('.')[0])

['submission_ensemble.csv', 'ValAcc79410.h5', 'submission_ensemble_aug_weighted2.csv', 'learnopencv8_ft_11_checkpoint.h5', 'submission_ensemble_aug.csv', 'y_pred_proba_resnet_aug.npy', 'densenet_top_1024_dropout_best4.h5', 'learnopencv9_ft_11_norot_checkpoint.h5', 'densenet_1024_dropout_best.h5', 'VGG-finetuning.ipynb', 'Untitled.ipynb', 'feature_extractor_linux.py', 'learnopencv_vgg4.h5', 'y_pred_proba_resnet.npy', 'learnopencv_vgg5.h5', 'Better resnet.ipynb', 'densenet_top_1024_dropout_best4_batch32.h5', 'learnopencv7_ft_11_checkpoint.h5', 'densenet_top_1024_dropout_best5_batch32.h5', 'y_pred_proba_densenet.npy', 'y_pred_proba_densenet_aug.npy', 'densenet_top_1024_dropout_best.h5', 'densenet_top_1024_dropout_best3.h5', 'transfer_learning.py', 'densenet_top_1024_dropout_best2.h5', 'run.sh', 'densenet_1024_dropout_best_test.h5', 'densenet_top_4000_1000_2.h5', 'densenet_top_1024_dropout_best6_batch32.h5', 'data_valid', 'learnopencv101_ft_checkpoint.h5', 'Untitled1.ipynb', 'classificatio

In [ ]:
from tqdm import tqdm
import os.path

def file_exists(fname):
    return os.path.isfile(fname) 

mode = 'valid'
target_shape = (7,7,1920)

the_ids = []
the_lables = []
n_classes = 128
for class_ in range(1, n_classes+1):
    print('Progress: ', class_, end='\r')
    data = pickle.load(open(FEATURES_LOCATION + F_CORE + mode + '_' + str(class_) + '.pkl', 'rb'))
    
    features = np.array(list(data.values()))
    ids = np.array([get_id_from_path(key) for key in list(data.keys())])
    labels = np.array([class_] * len(ids))
    
    for i in tqdm(range(len(ids))):
        if(file_exists('./data_'+mode+'/' + str(ids[i]) +'.npy')):
#             print('exists')
            continue
           
        if(features[i].shape == target_shape):
            np.save('./data_'+mode+'/' + str(ids[i]), features[i])
        else:
            print("Omitted file:", ids[i], '. Wrong shape.')
        
    the_ids.append(ids)
    the_lables.append(labels)
    
    del data
    del features
    del ids
    del labels

np.save('./data/meta/'+mode+'_ids', np.concatenate(the_ids))
np.save('./data/meta/'+mode+'_labels', np.concatenate(the_lables))
#     for i, id_ in enumerate(ids):
#         print("saved: ", i, end='\r')
#         np.savez_compressed('./data/' + str(id_), features = features[i])

In [ ]:
np.save('./data/meta/'+mode+'_ids', np.concatenate(the_ids))
np.save('./data/meta/'+mode+'_labels', np.concatenate(the_lables))

In [ ]:
np.save('./data_train/meta/'+mode+'_ids_copy', np.concatenate(the_ids))
np.save('./data_train/meta/'+mode+'_labels_copy', np.concatenate(the_lables))

### Source: https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly.html

In [2]:
import numpy as np
import keras

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=32, dim=(32,32,32), n_channels=1,
                 n_classes=10, shuffle=True, data_path = 'data/'):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.data_path = data_path
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels?)
        # Initialization
        if(self.n_channels and self.n_channels > 0):
            X = np.empty((self.batch_size, *self.dim, self.n_channels))
        else:
            X = np.empty((self.batch_size, *self.dim))
            
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            try:
                sample = np.load(self.data_path + str(ID) + '.npy')
                if(sample.shape == self.dim):            
                    # Store sample
                    X[i,] = np.load(self.data_path + str(ID) + '.npy')

                    if(self.labels):
                        y[i] = self.labels[ID]
            except:
                continue

        return X, keras.utils.to_categorical(y, num_classes=self.n_classes)

/home/1nnuser/lsdaenv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
model = keras.models.load_model('./final/ResNetTop_ValAcc7941.h5')

In [ ]:
model.summary()

In [ ]:
# Datasets
train_ids = np.load('./data/meta/train_ids.npy')
train_labels = np.load('./data/meta/train_labels.npy')

In [ ]:
print(train_ids.shape)
print(labels.shape)

In [ ]:
training_generator.labels

In [ ]:
# labels = np.delete(train_ids, [117531])
# # np.load('data/' + str(train_ids[117531]) + '.npy')
# np.save('./data/meta/train_ids', train_ids)
# np.save('./data/meta/train_labels', labels)

In [ ]:
for i, id_ in enumerate(train_ids[117520:]):
    print(i, id_, end='\r')
    sample = np.load('data_train/' + str(id_) + '.npy')
    if sample.shape[0] != 7 or sample.shape[1] != 7 or sample.shape[2] != 1920:
        print(sample.shape, id_)

In [ ]:
labels = {train_id: labels[i] for i, train_id in enumerate(train_ids)}

In [5]:
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
import keras

from keras import backend as K
K.clear_session()

n_classes = 128

# Parameters
train_params = {'dim': (7,7,1920),
          'batch_size': 32,
          'n_classes': n_classes,
          'n_channels': None,
          'shuffle': True,
          'data_path': 'data_train/'}

valid_params = {'dim': (7,7,1920),
          'batch_size': 10,
          'n_classes': n_classes,
          'n_channels': None,
          'shuffle': False,
          'data_path': 'data_valid/'}

# Datasets
train_ids = np.load('./data/meta/train_ids.npy')
train_labels = np.load('./data/meta/train_labels.npy')
train_labels = {id_: train_labels[i]-1 for i, id_ in enumerate(train_ids)}

valid_ids = np.load('./data/meta/valid_ids.npy')
valid_labels = np.load('./data/meta/valid_labels.npy')
valid_labels = {id_: valid_labels[i]-1 for i, id_ in enumerate(valid_ids)}

# Generators
training_generator = DataGenerator(train_ids, train_labels, **train_params)
validation_generator = DataGenerator(valid_ids, valid_labels, **valid_params)

# Design model
model = Sequential()
model.add(Flatten(name='flat', input_shape=(7,7,1920)))
# model.add(Dropout(0.1, name='flat_dropout'))
model.add(Dense(4000, activation='relu', name='hidden_4000'))
model.add(Dropout(0.2, name='hidden_4000_dropout'))
model.add(Dense(1000, activation='relu', name='hidden_1000'))
model.add(Dropout(0.2, name='hidden_1000_dropout'))
model.add(Dense(n_classes, activation='softmax', name='classification'))

In [6]:

# model = keras.models.load_model('densenet_top_1024_dropout_best.h5')

optim = keras.optimizers.Adam(lr=(1e-4))

# Compile the model
model.compile(loss='categorical_crossentropy',
            optimizer=optim,
            metrics=['acc'])

# Train model on dataset
model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    validation_steps=len(validation_generator)/2,
                    use_multiprocessing=True,
                    workers=6,
                    verbose=1,
                    callbacks=[keras.callbacks.ModelCheckpoint('densenet_top_4000_1000_2.h5', monitor='val_acc', verbose=2, save_best_only=True, mode='auto', period=1)],
                    epochs=3,
                    steps_per_epoch=400)

Epoch 1/3


ResourceExhaustedError: OOM when allocating tensor with shape[94080,4000] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: training/Adam/Variable/Assign = Assign[T=DT_FLOAT, _class=["loc:@training/Adam/Variable"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](training/Adam/Variable, training/Adam/zeros)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'training/Adam/Variable/Assign', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/1nnuser/lsdaenv/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/1nnuser/lsdaenv/lib/python3.5/site-packages/traitlet

In [ ]:
optim = keras.optimizers.Adam(lr=(2e-5))

# Compile the model
model.compile(loss='categorical_crossentropy',
            optimizer=optim,
            metrics=['acc'])

# Train model on dataset
history = model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    validation_steps=len(validation_generator)/2,
                    use_multiprocessing=True,
                    workers=6,
                    verbose=1,
                    callbacks=[keras.callbacks.ModelCheckpoint('densenet_top_4000_1000_4.h5', monitor='val_acc', verbose=2, save_best_only=True, mode='auto', period=1)],
                    epochs=3,
                    steps_per_epoch=400)

In [ ]:
model = keras.models.load_model('./densenet_top_4000_1000_2.h5')
print("MODEL LOADED")

In [ ]:
model.load_weights('./densenet_top_4000_1000_2.h5')

In [ ]:
optim = keras.optimizers.Adam(lr=(1e-5))

# Compile the model
model.compile(loss='categorical_crossentropy',
            optimizer=optim)
print("MODEL COMPILED")

# Train model on dataset
model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    validation_steps=len(validation_generator)/2,
                    use_multiprocessing=True,
                    workers=6,
                    verbose=1,
                    callbacks=[keras.callbacks.ModelCheckpoint('densenet_top_4000_1000_1-1.h5', monitor='val_acc', verbose=2, save_best_only=True, mode='auto', period=1)],
                    epochs=3,
                    steps_per_epoch=100)

In [ ]:
optim = keras.optimizers.Adam(lr=(2e-6))

# Compile the model
model.compile(loss='categorical_crossentropy',
            optimizer=optim,
            metrics=['acc'])

# Train model on dataset
model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    validation_steps=len(validation_generator)/2,
                    use_multiprocessing=True,
                    workers=6,
                    verbose=1,
                    callbacks=[keras.callbacks.ModelCheckpoint('densenet_top_4000_1000_5.h5', monitor='val_acc', verbose=2, save_best_only=True, mode='auto', period=1)],
                    epochs=3,
                    steps_per_epoch=400)

# Validation accuracy

In [ ]:
X_val, y_val = load_data('valid')

In [ ]:
y_pred = classification_model.predict(X_val)

In [ ]:
np.sum(np.argmax(y_pred, axis=1) + 1 == y_val) / len(y_val)

# ENSEMBLE

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

from keras.applications.densenet import DenseNet201 as densenet
from keras.applications.densenet import preprocess_input as densenet_pp

from keras.applications.resnet50 import ResNet50 as resnet
from keras.applications.resnet50 import preprocess_input as resnet_pp

DENSENET_IMG_SIZE = 224
RESNET_IMG_SIZE = 244

In [ ]:
ext_datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
)

ext_gen_flow = ext_datagen.flow_from_directory(
        '/home/1nnuser/data/test',
        target_size=(DENSENET_IMG_SIZE, DENSENET_IMG_SIZE),
        batch_size=32,
        class_mode=None
    )

In [ ]:
from keras.preprocessing import image
import glob

def load_test_images(directory='/home/1nnuser/data/test/images', img_size=224):
    files = glob.glob(directory + '/*')
    return files, np.array([image.img_to_array(image.load_img(file, target_size=(img_size, img_size))) for file in files])

In [ ]:
# Build a model by adding preprocessing before the pretrained CNN
def get_feature_extraction_model(model_name):
    if(model_name == 'densenet'):
        cnn_object, pp_function, img_size = densenet, densenet_pp, DENSENET_IMG_SIZE

    if(model_name == 'resnet'):
        cnn_object, pp_function, img_size = resnet, resnet_pp, RESNET_IMG_SIZE

    model = keras.models.Sequential()
    cnn_model = cnn_object(weights='imagenet', include_top=False)
    model.add(keras.layers.Lambda(pp_function, name='preprocessing', input_shape=(img_size, img_size, 3)))
    model.add(cnn_model)
    return model

In [ ]:
test_files, test_images_resnet = load_test_images(directory='/home/1nnuser/data/test/output/images', img_size = RESNET_IMG_SIZE)

In [ ]:
test_files, test_images_desenet = load_test_images(directory='/home/1nnuser/data/test/output/images', img_size = DENSENET_IMG_SIZE)

In [ ]:
extraction_model = get_feature_extraction_model('resnet')

In [ ]:
X = extraction_model.predict(test_images_resnet)
np.save('/home/1nnuser/data/resnet/test_features_aug', X)
np.save('/home/1nnuser/data/resnet/test_files_aug', test_files)

In [ ]:
extraction_model = get_feature_extraction_model('densenet')

In [ ]:
# X = extraction_model.predict_generator(ext_gen_flow, verbose=1, use_multiprocessing=True, workers=6)
X = extraction_model.predict(test_images_desenet)
np.save('/home/1nnuser/data/densenet/test_features_aug', X)
np.save('/home/1nnuser/data/densenet/test_files_aug', test_files)

In [ ]:
# def generate_data(directory, test_ids, batch_size):
#     i = 0
#     image_batch = []
#     for id_ in test_ids:
#         image = np.load(directory+str(id_)+'.npy')
#         image_batch.append(image)
#         if len(image_batch) >= batch_size:
#             yield np.array(image_batch)
#             image_batch = []
#     yield np.array(image_batch)
        
# def get_test_data(directory='../data_test_0/'):
#     files = glob.glob(directory+'*')
#     features = []
#     names = []
#     for i, file in enumerate(files):
#         print(i, end='\r')
#         features.append(np.load(file))
#         names.append(file)
#     return names, np.array(features)

In [ ]:
# names, features = get_test_data()

In [ ]:
# from subprocess import call

# def file_exists(fname):
#     return os.path.isfile(fname) 

# def _get_id_from_path(path):
#     return int(path.split('/')[-1].split('.')[0])

# aug_i = 0

# # call(['mkdir', '-p', '../data_test_'+str(aug_i)+'/'])

# print("Saving files.")
# for i, file in enumerate(test_files):
#     print(i+1, end='\r')
#     path_to_save = '../data_test_'+str(aug_i)+'/' + str(_get_id_from_path(file))
#     np.save(path_to_save, X[i])

# ids = [str(_get_id_from_path(file)) for file in test_files]
# np.save('../data/meta/test_ids', ids)

In [ ]:
# test_features = np.load('/home/1nnuser/data/densenet/test_features_aug.npy')
test_files_aug = np.load('/home/1nnuser/data/densenet/test_files_aug.npy')
test_features.shape, test_files.shape

In [ ]:
def get_id_from_aug_path(file_path):
    return file_path.replace('/home/1nnuser/data/test/output/images/images_original_', '').split('.jpg_')[0]

In [ ]:
test_files_aug_int = np.array([int(get_id_from_aug_path(path)) for path in test_files_aug])
test_files_indices_sorted_aug = np.argsort(test_files_aug_int)

In [ ]:
test_files_aug_int[test_files_indices_sorted_aug]

In [ ]:
test_files_reg = np.load('/home/1nnuser/data/densenet/test_files.npy')

In [ ]:
test_files_reg_int = np.array([int(get_id_from_path(path)) for path in test_files_reg])
test_files_indices_sorted_reg = np.argsort(test_files_reg_int)

In [ ]:
test_files_reg_int[test_files_indices_sorted_reg]

In [ ]:
# test_features = np.load('/home/1nnuser/data/resnet/test_features_aug.npy')
test_files_r = np.load('/home/1nnuser/data/resnet/test_files_aug.npy')
test_features.shape, test_files.shape

In [ ]:
test_files_r

In [ ]:
classification_model = keras.models.load_model('final/ResNetTop_ValAcc7941.h5')

In [ ]:
classification_model = keras.models.load_model('final/densenet_top_4000_1000_5.h5')

In [ ]:
y_pred_proba = classification_model.predict(test_features, verbose=1)

In [ ]:
np.save('y_pred_proba_resnet_aug', y_pred_proba)

In [ ]:
np.save('y_pred_proba_densenet_aug', y_pred_proba)

In [ ]:
y_pred_proba.shape

In [ ]:
y_proba_resnet = np.load('y_pred_proba_resnet.npy')
y_proba_resnet_aug = np.load('y_pred_proba_resnet_aug.npy')
y_proba_densenet = np.load('y_pred_proba_densenet.npy')
y_proba_densenet_aug = np.load('y_pred_proba_densenet_aug.npy')

In [ ]:
y_proba_resnet = y_proba_resnet[test_files_indices_sorted_reg]
y_proba_densenet = y_proba_densenet[test_files_indices_sorted_reg]
y_proba_resnet_aug = y_proba_resnet_aug[test_files_indices_sorted_aug]
y_proba_densenet_aug = y_proba_densenet_aug[test_files_indices_sorted_aug]

In [ ]:
y_proba_ensemble = y_proba_resnet + y_proba_densenet
y_proba_ensemble.shape

In [ ]:
y_proba_ensemble = 0.8 * y_proba_resnet + 0.6 * y_proba_densenet + 0.2 * y_proba_densenet_aug
y_proba_ensemble.shape

In [ ]:
y_pred = np.argmax(y_proba_ensemble, axis=1)+1
y_pred.shape, np.unique(y_pred)

In [ ]:
test_ids = [get_id_from_path(name) for name in test_files_reg[test_files_indices_sorted_reg]]
new_predictions = {}
for i, y_ in enumerate(y_pred):
    new_predictions[int(test_ids[i])] = y_

In [ ]:
len(list(new_predictions.keys()))

In [ ]:
from collections import Counter
counted = Counter(y_pred)
most_common_class = counted.most_common()[-1][0]
print(most_common_class)

replaced_counter = 0
for index in range(1, 12801):
    if(index not in new_predictions.keys()):
        new_predictions[index] = most_common_class
        replaced_counter+=1
        
print(replaced_counter)

In [ ]:
import csv
with open('submission_ensemble_aug_weighted.csv', 'r') as fp:
    reader = csv.reader(fp, delimiter=',', quotechar='"')
    # next(reader, None)  # skip the headers
    data_read = [row for row in reader]
    
data_read = np.array(data_read)
ids = data_read[1:,0]
predictions = data_read[1:,1]
ids = [int(id_.replace(' ', '')) for id_ in ids]
predictions = [int(prediction.replace(' ', '')) for prediction in predictions]
prev_predictions = {id_: prediction for id_, prediction in zip(ids, predictions)}

len(ids), len(predictions)

In [ ]:
np.sum([new_predictions[i] == prev_predictions[i] for i in range(1, len(prev_predictions.keys()))])

In [ ]:
new_predictions[3]

In [ ]:
ids = []
values = []
for key, value in new_predictions.items():
    ids.append(key)
    values.append(value)
    
out_dict = {}
out_dict['id'] = ids
out_dict['predicted'] = values

keys = sorted(out_dict.keys())
COL_WIDTH = 6
FMT = "%%-%ds" % COL_WIDTH

with open('submission_ensemble_aug_weighted2.csv', 'w') as csv:
    # Write keys    
    csv.write(','.join([k for k in keys]) + '\n')

    # Assume all values of dict are equal
    for i in range(len(out_dict[keys[0]])):
        csv.write(','.join([FMT % out_dict[k][i] for k in keys]) + '\n')

# OLD PART OF THE NOTEBOOK

In [ ]:
weights = np.ones_like(y) / (len(y))
_ = plt.hist(y, weights=weights, bins=np.max(y), alpha=0.7)
weights = np.ones_like(y_val) / (len(y_val))
_ = plt.hist(y_val, weights=weights, bins=np.max(y_val), alpha=0.5)

from collections import Counter
counted = Counter(y)
most_common_class = counted.most_common()[0][0]
        
print(counted[34])

In [ ]:
X = np.concatenate((X, X_val), axis=0)
y = np.concatenate((y, y_val), axis=0)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, stratify = y, test_size = 0.05)

In [ ]:
from sklearn.utils import class_weight
class_weight = class_weight.compute_class_weight('balanced',
                                             np.unique(y),
                                             y)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=0)
clf.fit(X, y)
clf.score(X_val, y_val)

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(1024,), max_iter=200, alpha=0.25,
                    solver='sgd', verbose=10, tol=1e-4, random_state=1,
                    learning_rate='adaptive', learning_rate_init=.1, )
mlp.fit(X, y)

In [ ]:
mlp.score(X_val, y_val)

In [ ]:
import keras
from keras import layers, optimizers, models
from keras.utils.np_utils import to_categorical
num_classes = 128

model = keras.models.Sequential()
model.add(layers.Dense(1024, activation='relu', name='hidden_1024', input_shape=(2048,)))
model.add(layers.Dropout(0.5, name='hidden_1024_dropout'))
model.add(layers.Dense(num_classes, activation='softmax', name='classification'))
print(model.layers[-3].get_weights())
model.load_weights('classification2_checkpoint.h5', by_name=True)
print(model.layers[-3].get_weights())

optim = keras.optimizers.Adam(lr=(1e-3))
# optim = keras.optimizers.RMSprop(lr=0.001)
# optim = keras.optimizers.SGD(lr=1e-4, momentum=0.9, nesterov=True)

# Compile the model
model.compile(loss='categorical_crossentropy',
            optimizer=optim,
            metrics=['acc'])

In [ ]:
y_preds = np.argmax(model.predict(X_val), axis=1) + 1
np.sum(y_preds == y_val) / len(y_val)

In [ ]:
model.layers[-3].get_weights()

In [ ]:
y_cat = to_categorical(y-1, num_classes)
y_val_cat = to_categorical(y_val-1, num_classes)

In [ ]:
model.fit(X, y_cat, batch_size = 200, validation_data=(X_val, y_val_cat), epochs=10,
         callbacks=[keras.callbacks.ModelCheckpoint('classification2_checkpoint.h5', monitor='val_acc', verbose=2, save_best_only=True, mode='auto', period=1)])

In [ ]:
# from keras.applications.densenet import DenseNet201, DenseNet121
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Dense
from keras import layers, optimizers, models
import keras
from keras import backend as K
# from keras.applications.densenet import preprocess_input
import time

from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input
import matplotlib.pyplot as plt

from keras.models import load_model
from keras.applications import imagenet_utils
from keras.applications.imagenet_utils import _preprocess_symbolic_input
from keras.preprocessing import image

In [ ]:
target_size = (224,224)
# validation_dir = './data/valid'
# train_dir = './data/train'
validation_dir = '/furniture-data/valid'
train_dir = '/furniture-data/train'

train_datagen = ImageDataGenerator(
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True,
        preprocessing_function=preprocess_input
      )
 
validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
 
# Change the batchsize according to your system RAM
train_batchsize = 32
val_batchsize = 10
 
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=target_size,
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=target_size,
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

In [ ]:
cnn = ResNet50(weights='imagenet', include_top=False)

for layer in cnn.layers:
    layer.trainable = False

In [ ]:
# create the base pre-trained model
model = keras.models.Sequential()
# model.add(keras.layers.Lambda(preprocess_input, name='preprocessing', input_shape=(224, 224, 3)))
model.add(keras.layers.InputLayer(input_shape=(224,224,3)))
model.add(cnn)
model.add(keras.layers.Reshape(target_shape=(2048,)))

In [ ]:
model.summary()
import h5py
features = []
labels = []
batches = 0
for x, y in train_generator:
    print(batches, end='\r')
    features.append(model.predict(x))
    labels.append(np.argmax(y, axis=1)+1)
    batches += 1
    if batches >= 1000:
        h5f = h5py.File('data.h5', 'w')
        h5f.create_dataset('features', data=features)
        h5f.create_dataset('labels', data=labels)
        h5f.close()
    

In [ ]:
h5f = h5py.File('data.h5','r')
features = h5f['features'][:]
labels = h5f['labels'][:]
h5f.close()

In [ ]:
import keras
from keras import layers, optimizers, models
from keras.utils.np_utils import to_categorical
num_classes = 128

model = keras.models.Sequential()
model.add(layers.Dense(1024, activation='relu', name='hidden_1024', input_shape=(2048,)))
model.add(layers.Dropout(0.5, name='hidden_1024_dropout'))
model.add(layers.Dense(num_classes, activation='softmax', name='classification'))
# print(model.layers[-3].get_weights())
# model.load_weights('classification2_checkpoint.h5', by_name=True)
# print(model.layers[-3].get_weights())

optim = keras.optimizers.Adam(lr=(1e-3))
# optim = keras.optimizers.RMSprop(lr=0.001)
# optim = keras.optimizers.SGD(lr=1e-4, momentum=0.9, nesterov=True)

# Compile the model
model.compile(loss='categorical_crossentropy',
            optimizer=optim,
            metrics=['acc'])

In [ ]:
for e in range(1,30):
    for x_batch, y_batch in zip(features,labels):
        model.fit(x_batch, y_batch)
#             samples = 0
#             for x_batch, y_batch in val_datagen:
#                 y_pred = np.argmax(model.predict(X_val), axis=1) + 1
#                 correct = np.sum(y_pred == y_batch)
#                 samples += x_batch.shape[0]
#             print("val acc: ", correct / samples)
#             break
model.save_weights('test.h5')

In [ ]:
model_classification = keras.models.load_model('./ValAcc79410.h5')
model_classification.summary()

In [ ]:
# create the base pre-trained model
model = keras.models.Sequential()
# model.add(keras.layers.Lambda(preprocess_input, name='preprocessing', input_shape=(224, 224, 3)))
model.add(keras.layers.InputLayer(input_shape=(224,224,3)))
model.add(cnn)
# model.add(keras.layers.Reshape(target_shape=(2048,)))
model.add(keras.layers.Flatten(name='flat'))
model.add(layers.Dense(1024, activation='relu', name='hidden_1024', input_shape=(2048,)))
model.add(layers.Dropout(0.5, name='hidden_1024_dropout'))
model.add(layers.Dense(128, activation='softmax', name='classification'))
# print(model.layers[-3].get_weights())
# print(model.layers[-3].get_weights())
# model.add(model_classification)

In [ ]:
model.layers

In [ ]:
for layer in model.layers[1].layers[-11:]:
    layer.trainable = True
    print(layer.name, layer.trainable)
model.load_weights('./learnopencv9_ft_11_norot_checkpoint.h5', by_name=True)


In [ ]:
for layer in model.layers[1].layers[-35:]:
    layer.trainable = True
    print(layer.name, layer.trainable)


In [ ]:
# print(model.layers[-3].get_weights())
classification_model = keras.models.load_model('./ValAcc79410.h5')
# model.layers[-3].get_weights()

In [ ]:
classification_model.summary()

In [ ]:
cnn = ResNet50(weights='imagenet', include_top=False)

In [ ]:
# for layer in cnn.layers:
#     layer.trainable = False

# define new model
model = keras.models.Sequential()
model.add(keras.layers.InputLayer(input_shape=(224,224,3)))
# model.add(keras.layers.Lambda(preprocess_input, name='preprocessing', input_shape=(224, 224, 3)))

# append all VGG16 layers
# for layer in cnn.layers: 
#     model.add(layer)
    
model.add(cnn)

# append the new top model
for layer in classification_model.layers:
    model.add(layer)
    
# we do not want to retrain the whole new network (very time-consuming!). 
# Instead, we will only fine-tune the last layers ...
# for i in range(0, len(model.layers)):
#     if i < 15:
#         print("%i: Setting layer %s to non-trainable" % (i, str(model.layers[i])))
#         model.layers[i].trainable=False
#     else:
#         print("%i: Keeping layer %s trainable" % (i, str(model.layers[i])))                    
# print(model.summary())

In [ ]:
for imgs, labels in validation_generator:
    y_preds = np.argmax(model.predict(imgs), axis=1) + 1
    print(np.sum(y_preds == labels) / len(labels))

In [ ]:
for layer in model.layers[1].layers[-11:]:
    layer.trainable = True
    print(layer.name, layer.trainable)

# if load:

# Show a summary of the model. Check the number of trainable parameters
for layer in model.layers:
    print(layer, layer.trainable)

for layer in model.layers[1].layers:
    print(layer, layer.trainable)

model.layers[1].summary()
model.summary()

print(model.layers[-3].get_weights())

In [ ]:
optim = keras.optimizers.Adam(lr=(1e-4))

# Compile the model
model.compile(loss='categorical_crossentropy',
            optimizer=optim,
            metrics=['acc'])

print(model.layers[-3].get_weights())

# Train the model
history = model.fit_generator(
        train_generator,
        #   steps_per_epoch=train_generator.samples/train_generator.batch_size ,
        steps_per_epoch=200,
        epochs=30,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples/validation_generator.batch_size,
        verbose=1,
        callbacks=[keras.callbacks.ModelCheckpoint('learnopencv101_ft_checkpoint.h5', monitor='val_loss', verbose=2, save_best_only=True, mode='auto', period=1)]
    )

# # here's a more "manual" example
# for e in range(epochs):
#     print('Epoch', e)
#     batches = 0
#     for x_batch, y_batch in train_datagen:
#         np.apply_along_axis(preprocess_input, 1, x_batch)
#         model.fit(x_batch, y_batch)
#         batches += 1
#         if batches >= len(x_train) / train_batchsize:
#             samples = 0
#             for x_batch, y_batch in val_datagen:
#                 y_pred = np.argmax(model.predict(X_val), axis=1) + 1
#                 correct = np.sum(y_pred == y_batch)
#                 samples += x_batch.shape[0]
#             print("val acc: ", correct / samples)
#             break

# PREDICTIONS AND SAVING THEM

In [ ]:
# ONE PREDICTION AT A TIME
predictions = {}
for index, feature_vector in X_test.items():
    predictions[int(index)] = the_clf.predict(feature_vector)[0]
    print(index, end='\r')

In [ ]:
# BATCH OF PREDICTIONS
X_test_arr = np.array(list(X_test.values()))
X_test_arr = X_test_arr.reshape(-1,2048)

y_pred = the_clf.predict(X_test_arr)
predictions2 = {}
for i, index in enumerate(X_test.keys()):
    predictions2[int(index)] = y_pred[i]

In [ ]:
prev_pred = deepcopy(predictions)
predictions = predictions2

In [ ]:
from collections import Counter
counted = Counter(predictions.values())
most_common_class = counted.most_common()[0][0]

for index in range(1, 12801):
    if(index not in predictions.keys()):
        predictions[index] = most_common_class

In [ ]:
ids = []
values = []
for key, value in predictions.items():
    ids.append(key)
    values.append(value)
    
out_dict = {}
out_dict['id'] = ids
out_dict['predicted'] = values

In [ ]:
keys = sorted(out_dict.keys())
COL_WIDTH = 6
FMT = "%%-%ds" % COL_WIDTH

with open('try_this_one.csv', 'w') as csv:
    # Write keys    
    csv.write(','.join([k for k in keys]) + '\n')

    # Assume all values of dict are equal
    for i in range(len(out_dict[keys[0]])):
        csv.write(','.join([FMT % out_dict[k][i] for k in keys]) + '\n')

In [ ]:
# X_train, X_val, y_train, y_val = train_test_split(X, y, stratify = y, test_size = 0.1)
warm_mlp = MLPClassifier(hidden_layer_sizes=(200,), max_iter=1, alpha=1e-4,
            solver='sgd', verbose=10, tol=1e-4, random_state=None,
            learning_rate='adaptive', learning_rate_init=.001,
            warm_start=True)

val_scores = []
losses = []
clfs = []
for i in range(1,4000):
    warm_mlp.fit(X_train,y_train)
    val_score = warm_mlp.score(X_val, y_val)
    
    val_scores.append(val_score)
    losses.append(warm_mlp.loss_)
    if(val_score > 0.7):
        clfs.append(deepcopy(warm_mlp))
    
    print("Val score:", val_score)

from sklearn.externals import joblib
joblib.dump(clfs, 'clfs_250features.pkl') 

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# test_datagen = ImageDataGenerator(rescale=1./255)

# train_generator = train_datagen.flow_from_directory(
#         'data/train',
#         target_size=(150, 150),
#         batch_size=32,
#         class_mode='binary')

# validation_generator = test_datagen.flow_from_directory(
#         'data/validation',
#         target_size=(150, 150),
#         batch_size=32,
#         class_mode='binary')

# train_generator.flow(x, y=None, batch_size=32, shuffle=True, seed=None, save_to_dir='./gen_test', save_prefix='', save_format='png', subset=None)
iterator = train_datagen.flow_from_directory('data/valid', target_size=(244, 244), color_mode='rgb', classes=None, class_mode='categorical', batch_size=32, shuffle=True, 
                                    seed=None, save_to_dir='gen_test', save_prefix='', save_format='png', follow_links=False, subset=None, interpolation='nearest')

for i in iterator:
    print(i)